## Test Flask App
From the another notebook (such as `run_flask.ipynb`), you can run the flask app in a cell and test it from here using python http or `curl` statements.

```python
# run_flask.ipynb
!FLASK_APP=wsgi.py flask run
```

In [ ]:
!curl http://localhost:5000/status

In [ ]:
!curl -X POST -H "Content-Type: application/json" --data "@sample-requests/twodogs.json" http://localhost:5000/predictions

In [ ]:
import requests
import json
import pprint

with open ("sample-requests/twodogs.json", "r") as f:
  data = json.load(f)
    
pp = pprint.PrettyPrinter(width=41, compact=True)  
response = requests.post('http://127.0.0.1:5000/predictions', json.dumps(data))

pp = pprint.PrettyPrinter(width=41, compact=True)
last = len(response.json()['detections'])
pp.pprint(response.json()['detections'][0])
pp.pprint(response.json()['detections'][last-1])

Now that our application is running and making predictions as expected, you can go back to the workshop instructions to see how we integrate this into an App with a user interface. 